In [18]:
import sys
import os

sys.path.append(os.path.abspath("../scripts"))
from scrape_reviews import ScrapeReview

app_ids = ["com.combanketh.mobilebanking", "com.dashen.dashensuperapp", "com.boa.boaMobileBanking"]
scraper = ScrapeReview(
    app_ids=app_ids,
    save=True,
    scrape_metadata=False,
    max_retries=5,
    delay=1.5
)

reviews_df = scraper.get_reviews()
metadata = scraper.get_metadata()

print(f"Scraped {len(reviews_df)} reviews")
print(f"Scraped metadata for {len(metadata)} apps")

Scraping reviews for 3 apps...


Scraping apps:   0%|          | 0/3 [00:00<?, ?it/s]

No reviews scraped for com.combanketh.mobilebanking


Scraping apps:  33%|███▎      | 1/3 [00:23<00:47, 23.69s/it]

No reviews scraped for com.dashen.dashensuperapp


Scraping apps:  67%|██████▋   | 2/3 [00:43<00:21, 21.46s/it]

No reviews scraped for com.boa.boaMobileBanking


Scraping apps: 100%|██████████| 3/3 [01:01<00:00, 20.40s/it]

Scraped 0 reviews
Scraped metadata for 0 apps
